In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
from easyocr import Reader

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/apuru/OneDrive/Desktop/Project final/yolov5/runs/train/exp7/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\apuru/.cache\torch\hub\master.zip
YOLOv5  2023-5-19 Python-3.9.4 torch-1.13.0+cpu CPU

Fusing layers... 


requirements: C:\Users\apuru\AppData\Local\Programs\Python\Python39\Lib\site-packages\requirements.txt not found, check failed.


Model summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [4]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("fir-project-13301-firebase-adminsdk-q6vld-3cda7718e6.json")
firebase_admin.initialize_app(cred)

db=firestore.client()

In [5]:
data_ref = db.collection(u'user')
docs = data_ref.stream()
t=[]
for doc in docs:
    if doc.id == "123":
        ans = doc.to_dict()
        t=(ans['numberplates'])
        print(ans);
print(t)
numberplates=t

{'Demo': [{'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 4, 30, 5, 22, 45, 319000, tzinfo=datetime.timezone.utc), 'Numberplate': 'AP1234BC23'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 53, 28, 621000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'Out', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 54, 19, 814000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 10, 58, 976034, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED45

In [6]:
numberplates

['TS10ED4584']

In [7]:
from datetime import datetime

# Getting the current date and time
dt = datetime.now()


In [8]:
def check(actual,real):
    if(actual == None):
        return len(real)
    else:
        actual=actual.upper()
        real=real.upper()
    n1=len(actual)
    n2=len(real)
    if n1<n2:
        n=n1
    else:
        n=n2
    cnt=0
    for i in range(n):
        if actual[i] == real[i]:
            continue
        else:
            cnt=cnt+1
    return cnt

In [12]:
import enchant
from datetime import datetime
user_ref = db.collection(u'user')
historical = user_ref.stream()
t=[]
global last
last =""
for doc in historical:
    ans2 = doc.to_dict()
    if ans2['id'] == 123:
        res=ans['Demo']
        document_id=doc.id
        break

    
# Getting the current date and time
dt = datetime.now()
#actual = 'AP1234BC23'
def exit_detection(last):
    def most_common(lst):
        return max(set(lst), key=lst.count, default=None)

    #cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(0)
    reader = Reader(['en'])
    mylist = []
    flag = False
    while cap.isOpened():
        ret, frame = cap.read()

        txt = ""

        # now = datetime.now()
        # Make detections
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if ret == False:
            break
        results = model(frame)
        cv2.imshow('YOLO', np.squeeze(results.render()))
        if not results:
            continue
        imgt = results.xyxy[0]
        i = list(imgt)
        l = []
        for j in i:
            temp = j.cpu().numpy()
            l.append(temp)
        if not l or len(l[0]) < 4:
            continue
        m = l[0][4]
        pos = 0
        for j in range(1, len(l)):
            if l[j][4] > m:
                m = l[j][4]
                pos = i
        if (m < 0.85):
            continue
        xmin = int(l[pos][0])
        ymin = int(l[pos][1])
        xmax = int(l[pos][2])
        ymax = int(l[pos][3])
        # img1=mpimg.imread(frame)
        modimg = frame[ymin:ymax, xmin:xmax, :]
        ans = reader.readtext(modimg)

        #print(ans)
        if (not ans or len(ans) < 2 or len(ans[1]) < 2):
            continue
        else:
            for i in range(len(ans)):
                if ans[i][1] == 'IND' or 'ND' in ans[i][1]:
                    continue
                txt = txt + ans[i][1]
            if (flag == False):
                mylist.append(txt)
        if (len(mylist) > 10):
            actual = most_common(mylist)
            actual = accurate(actual)
            flag = True
            print(actual)
            last=exit_update(actual,last)
            
                    
        if (flag == True):
            if(actual!=None):
                if (check(actual, txt) > 5):
                    mylist = []
                    flag = False
            else:
                print("none")

def accurate(actual):
    #print(type(actual))
    #print(numberplates)
    correct=[x for x in actual if x!=' ']
    actual=""
    for i in correct :
        actual+=i
    #print(actual)
    for i in numberplates:
        if(check(actual, i) <=2):
            return i
    
def exit_update(actual,last):
        if last!=actual and actual!=None:
            numberplates_ref = db.collection(u'numberplate')
            user_ref = db.collection(u'user').document(document_id)

            query1= numberplates_ref.where('numberplate',"==",actual).where('status',"==",'inside')

            docs1=query1.get()
            print(docs1)
            if docs1 == [] : 
                ans1={
                    'Moved' : 'Out',
                    'Numberplate': actual,
                    'Time' : dt,
                    'access' : 'unauthorized'
                }
                res.append(ans1)
                print(res)
                user_ref.update({
                    u'Demo' : res,
                })
                last=actual
                print('Update succesful')
                print('UnAuthorized')
                
            else:
                for doc in docs1 :
                    doc.reference.update({"status":"outside", "outtime":dt})
                    print('Update sucessful')
                    ans1={
                        'Moved' : 'Out',
                        'Numberplate': actual,
                        'Time' : dt,
                        'access' : 'authorized'
                    }
                    res.append(ans1)
                    print(res)
                    user_ref.update({
                        u'Demo' : res,
                    })
                    print('Update succesful')
                    print('Authorized')
                    last=actual

        return last
exit_detection(last)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


TS10ED4584
[]
[{'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 4, 30, 5, 22, 45, 319000, tzinfo=datetime.timezone.utc), 'Numberplate': 'AP1234BC23'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 53, 28, 621000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'Out', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 54, 19, 814000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 10, 58, 976034, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS1

In [14]:
def entry_detection(last):
    def most_common(lst):
        return max(set(lst), key=lst.count, default=None)

    cap = cv2.VideoCapture(0)
    #cap = cv2.VideoCapture('http://192.168.29.157:81/stream')
    reader = Reader(['en'])
    mylist = []
    flag = False
    while cap.isOpened():
        ret, frame = cap.read()

        txt = ""

        # now = datetime.now()
        # Make detections
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if ret == False:
            break
        results = model(frame)
        cv2.imshow('YOLO', np.squeeze(results.render()))
        if not results:
            continue
        imgt = results.xyxy[0]
        i = list(imgt)
        l = []
        for j in i:
            temp = j.cpu().numpy()
            l.append(temp)
        if not l or len(l[0]) < 4:
            continue
        m = l[0][4]
        pos = 0
        for j in range(1, len(l)):
            if l[j][4] > m:
                m = l[j][4]
                pos = i
        if (m < 0.85):
            continue
        xmin = int(l[pos][0])
        ymin = int(l[pos][1])
        xmax = int(l[pos][2])
        ymax = int(l[pos][3])
        # img1=mpimg.imread(frame)
        modimg = frame[ymin:ymax, xmin:xmax, :]
        ans = reader.readtext(modimg)

        #print(ans)
        if (not ans or len(ans) < 2 or len(ans[1]) < 2):
            continue
        else:
            for i in range(len(ans)):
                if ans[i][1] == 'IND' or 'ND' in ans[i][1]:
                    continue
                txt = txt + ans[i][1]
            if (flag == False):
                mylist.append(txt)
        if (len(mylist) > 10):
            actual = most_common(mylist)
            actual = accurate(actual)
            print(actual)
            flag = True
            last=entry_update(actual,last)  
            
                    
        if (flag == True):
            if(actual!=None):
                if (check(actual, txt) > 5):
                    mylist = []
                    flag = False
            else:
                print("none")


def accurate(actual):
    #print(type(actual))
    #print(numberplates)
    correct=[x for x in actual if x!=' ']
    actual=""
    for i in correct :
        actual+=i
    #print(actual)
    for i in numberplates:
        if(check(actual, i) <=2):
            return i
    

def entry_update(actual,last):
    if last!=actual:
        numberplates_ref = db.collection(u'numberplate')
        user_ref = db.collection(u'user').document(document_id)

        query1= numberplates_ref.where('numberplate',"==",actual).where('status',"==",'outside')

        docs1=query1.get()
        print(docs1)
        if docs1 == [] : 
            ans1={
                'Moved' : 'In',
                'Numberplate': actual,
                'Time' : dt,
                'access' : 'unauthorized'
            }
            res.append(ans1)
            print(res)
            user_ref.update({
                u'Demo' : res,
            })
            last=actual
            print('Update succesful')
            print('UnAuthorized')
            return last
        else:
            for doc in docs1 :
                doc.reference.update({"status":"inside", "outtime":dt})
                print('Update sucessful')
                ans1={
                    'Moved' : 'In',
                    'Numberplate': actual,
                    'Time' : dt,
                    'access' : 'authorized'
                }
                res.append(ans1)
                print(res)
                user_ref.update({
                    u'Demo' : res,
                })
                print('Update succesful')
                print('Authorized')
                last=actual
                return last
last=""
entry_detection(last)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


None
[]
[{'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 4, 30, 5, 22, 45, 319000, tzinfo=datetime.timezone.utc), 'Numberplate': 'AP1234BC23'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 53, 28, 621000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'Out', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 7, 4, 54, 19, 814000, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'authorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 10, 58, 976034, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED4584'}, {'Moved': 'In', 'access': 'unauthorized', 'Time': DatetimeWithNanoseconds(2023, 5, 18, 21, 17, 10, 541413, tzinfo=datetime.timezone.utc), 'Numberplate': 'TS10ED458

In [18]:
def check(actual,real):
    if(actual == None):
        return len(real)
    else:
        actual=actual.upper()
        real=real.upper()
    n1=len(actual)
    n2=len(real)
    if n1<n2:
        n=n1
    else:
        n=n2
    cnt=0
    for i in range(n):
        if actual[i] == real[i]:
            continue
        else:
            cnt=cnt+1
    return cnt